BLEu Score 계산을 위한 라이브러리 업데이트
번역한 문장이 얼마나 유사한지를 평가해주는 평가 척도

In [137]:
!pip install torchtext==0.6.0

spaCY 라이브러리 : Tokenization, tagging 을 위한

In [138]:
%%capture
!python -m spacy download en

In [139]:
%%capture
!python -m spacy download de

In [140]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰
spacy_de = spacy.load('de_core_news_sm') # 독일어 토큰

In [141]:
# spacy객체를 생성해 간단한 토큰화
tokenized= spacy_en.tokenizer('I am a graduate student.')

for i, token in enumerate(tokenized):
    print(f' {i} : {token.text}')

 0 : I
 1 : am
 2 : a
 3 : graduate
 4 : student
 5 : .


In [142]:
# 영어 독일어를 토큰화 함수 정의
def tokenized_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

def tokenized_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

Field 라이브러리를 통해 데이터셋에 대한 구체적인 전처리 내용 명시


In [168]:
from torchtext.data import Field, BucketIterator

# Source 독일어 , Target 영어
# 단어의 처음과 끝은 sos, eos, 모두 소문자로 변환, Transform함수의 batch를 위해 batch를 맨앞으로 True
SRC = Field(tokenize=tokenized_de, init_token='<sos>', eos_token='<eos>', lower=True, batch_first=True)
TRG = Field(tokenize=tokenized_en, init_token='<sos>', eos_token='<eos>', lower=True, batch_first=True)

In [169]:
# Multi30k 를 불러오기, SRC,TRG 전처리를 사용
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(root='./data',exts=(".de",".en"), fields=(SRC,TRG))


downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:03<00:00, 392kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 97.4kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 92.9kB/s]


In [145]:
print(f'train set length : {len(train_dataset)}')
print(f'valid set length: {len(valid_dataset)}')
print(f'test dataset length: {len(test_dataset)}')

print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

train set length : 29000
valid set length: 1014
test dataset length: 1000
['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


field 객체의 build_vocab 메서드를 통해 영어와 독어의 단어 사전 생성   
-최소 2번이상의 단어만을 선택, input의 demension 크기 알 수 있음

In [146]:
SRC.build_vocab(train_dataset,min_freq=2)
TRG.build_vocab(train_dataset,min_freq=2)

print(f'len(SRC) : {len(SRC.vocab)}')
print(f'len(TRG) : {len(TRG.vocab)}')

len(SRC) : 7853
len(TRG) : 5893


In [147]:
print(TRG.vocab.stoi[TRG.pad_token]) #unknown token
print(TRG.vocab.stoi['<sos>'])
print(TRG.vocab.stoi['<eos>'])
print(TRG.vocab.stoi['hello'])

1
2
3
4112


한문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력 되어야합니다.   
- 하나의 배치에 포함된 문장들이 가지는 단어개수가 유사하도록 만들고   
- 이를 Bucketlterator를 사용한다

In [148]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

# Dataloder 와 유사한 사용
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size = BATCH_SIZE,
    device=device
)

In [149]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f'첫번째 배치의 크기 : {src.shape}')
    print(f'문장의 개수 : {src.shape[1]}')

    # 현재 배치에 있는 0번 문장에 대한 정보 출력
    # 첫번쨰 index0 = 2 : sos, index16 = 3 : eos로 문장 끝
    for i in range(src.shape[1]):
        print(f'index {i} : {src[0][i].item()}')
    break

첫번째 배치의 크기 : torch.Size([128, 29])
문장의 개수 : 29
index 0 : 2
index 1 : 5
index 2 : 177
index 3 : 25
index 4 : 0
index 5 : 127
index 6 : 1475
index 7 : 7
index 8 : 14
index 9 : 1521
index 10 : 4
index 11 : 3
index 12 : 1
index 13 : 1
index 14 : 1
index 15 : 1
index 16 : 1
index 17 : 1
index 18 : 1
index 19 : 1
index 20 : 1
index 21 : 1
index 22 : 1
index 23 : 1
index 24 : 1
index 25 : 1
index 26 : 1
index 27 : 1
index 28 : 1


Multi Head Attention 아키텍쳐   
- Queriy, key, value를 받는다. 모두 같은 차원   
   
- Hyperparameter   
-- hidden_dim : 하나의 단어에 대한 임베딩 차원   
-- n_heads : head의 개수   


In [150]:
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_rate, device):
        super().__init__()

        assert hidden_dim % n_heads==0
        
        self.hidden_dim = hidden_dim
        self.n_heads = n_heads
        self.head_dim = hidden_dim // n_heads

        self.fc_Q = nn.Linear(hidden_dim, hidden_dim) # Query
        self.fc_K = nn.Linear(hidden_dim, hidden_dim) # Key
        self.fc_V = nn.Linear(hidden_dim, hidden_dim) # value 계산 FC layer

        self.fc_o = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout_rate)
        # Q,K,V에 해당하는 값에 루트 한 값을 나누어 주어서 softmax에 되도록
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value , mask =None):
        batch_size = query.shape[0]

        Q = self.fc_Q(query)
        K = self.fc_K(key)
        V = self.fc_V(value)
        # h개가 각각 하나씩 들어가도록
        Q = Q.view(batch_size,-1, self.n_heads,self.head_dim).permute(0,2,1,3)
        K = K.view(batch_size,-1, self.n_heads,self.head_dim).permute(0,2,1,3)
        V = V.view(batch_size,-1, self.n_heads,self.head_dim).permute(0,2,1,3)

        # attention Energy 계산
        energy = torch.matmul(Q,K.permute(0,1,3,2)) / self.scale
        # energy = [batch_size, n_heads, query_len, key_len]

        # 결과값이 0인 부분을 아주 작은 값으로 채우기
        if mask is not None:
            energy = energy.masked_fill(mask==0, -1e10)
        # attention 계산, 각 단어에 대한 확률값

        attention = torch.softmax(energy, dim= -1)
        # attention = [batch_size, n_heads, query_len, key_len]

        x = torch.matmul(self.dropout(attention), V)
        # x = [batch_size, n_heads, query_len, key_len]

        x = x.permute(0,2,1,3).contiguous()
        # x = [batch_size, query_len, n_heads, head_dim]
        
        x = x.view(batch_size,-1,self.hidden_dim)
        # x = [batch_size, query_len, hidden_dim]

        x = self.fc_o(x)
        # x= [batch_size, query_len, hidden_dim]
        return x, attention

Position-wise Feedforward
- 입력과 출력의 차원이 동일 합니다   
-- hidden_dim : 하나의 단어에 대한 임베딩 차원   
-- pf_dim : Feedforward 레이어에서의 내부 임베딩 차원   

In [151]:
class PositionwiseFeedforward(nn.Module):
    def __init__(self, hidden_dim, pf_dim, dropout_rate):    
        super().__init__()

        self.fc1 = nn.Linear(hidden_dim, pf_dim)
        self.fc2 = nn.Linear(pf_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self,x):

        x = self.dropout(x)

        x = self.fc1(x)
        x= self.fc2(x)
        return x 

Encode layer    
-- 입력과 출력의 차원이 같습니다.   
-- 인코더 레이어를 여러번 중첩해 사용한다.
- 파라미터   
-- hidden_dim : 하나의 단어에 대한 임베딩 차원   
-- n_head : head의 개수   
-- pf_dim : Feedforward 레이어에서의 내부 임베딩 차원   
<pad> 토큰에 대하여 mask값을 0으로 설정합니다 

In [152]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_rate,device):
        super().__init__()

        self.self_attention_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttention(hidden_dim, n_heads, dropout_rate,device)
        self.position_feedforward = PositionwiseFeedforward(hidden_dim, pf_dim, dropout_rate)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, src, src_mask):
        # src = [batch_size, src_len, hidden_dim]
        # src_mask = [batch_size, src_len]

        # self.attention
        _src, _ = self.self_attention(src,src,src,src_mask)

        # dropout, residual connection and layer norm
        src = self.self_attention_norm(src+self.dropout(_src))

        # src = [batch_size, src_len, hidden_dim]

        # position
        _src = self.position_feedforward(src)

        src = self.ff_layer_norm(src + self.dropout(_src))

        return src

전체 Encoder    
- 전체 인코더를 정의합니다.   
-- input dim : 하나의 단어에 대한 원 핫 인코딩 차원   
-- hidden_dim : 하나의 단어에 대한 임베딩 차원   
-- n_layer : 내부적으로 사용할 인코더 레이어의 개수   
-- n_head : head의 개수   
-- pf_dim : Feedforward 레이어에서의 내부 임베딩 차원   
-- max_length : 문장 내 최대 단어의 개수   
- positional embedding도 학습할 수 있음. BERT와 같은 모델에서 사용되는 방법

In [153]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_rate, device, max_length=100):
        super().__init__()

        self.device = device
        self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
        # 위치 position vector도 학습
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)
        # n_layer 개수 만큼 반복
        self.layers = nn.ModuleList([EncoderLayer(hidden_dim,n_heads,pf_dim,dropout_rate, device) for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout_rate)
        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, src,src_mask):
        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0,src_len).unsqueeze(0).repeat(batch_size,1).to(self.device)
        # pos = [batch_size, src_len]

        # 소스 문장의 임베딩과 위치 임베딩을 더함
        src = self.tok_embedding(src) * self.scale
        
        src = self.dropout(src)
        
        # 모든 레이어를 지나가면서 계산
        for layer in self.layers:
            src = layer(src, src_mask)
        
        return src

Decoder layer
- 입력과 출력 차원이 같다   
- 디코더 레이어를 여러번 중첩해서 사용   
- 디코더 레이어는 두개의 Multi-Head attention이 사용됨   
-- hidden_dim : 하나의 단어에 대한 임베딩 차원
-- n_heads : head의 개수   
-- pd_dim : Feedforward 레이어 에서 내부 임베딩 차원
- 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용 

In [154]:
class Decoderlayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_rate, device):
        super().__init__()

        self.self_attention_norm = nn.LayerNorm(hidden_dim)
        self.encode_atten_norm = nn.LayerNorm(hidden_dim)
        self.feedfor_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttention(hidden_dim,n_heads,dropout_rate,device)
        self.encoder_attention = MultiHeadAttention(hidden_dim,n_heads,dropout_rate,device)
        self.position_feedforward = PositionwiseFeedforward(hidden_dim,pf_dim, dropout_rate)
        self.dropout = nn.Dropout(dropout_rate)
    
    
    def forward(self, trg, enc_src, trg_mask, src_mask):
        # encoder의 출력값enc_src를 attention 하는 구조

        # trg =[ batch_size, trg_len, hidden_dim]
        # enc_src = [batch_size, src_len, hidden_dim]
        # trg_mask = [batch_size, trg_len]
        # src_mask = [batch_size, src_len]

        # self attention
        _trg, _ = self.self_attention(trg,trg,trg,trg_mask)
        _trg = self.dropout(_trg)
        trg = self.self_attention_norm(trg + _trg)

        # trg = [batch_size, trg_len, hidden_dim]

        # encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        _trg = self.dropout(_trg)
        trg = self.encode_atten_norm(trg+_trg)

        # position feedforward
        _trg = self.position_feedforward(trg)
        _trg = self.dropout(_trg)
        trg = self.feedfor_layer_norm(trg + _trg)

        # trg = [batch_size, trg_len, hidden_dim]
        # attention = [batch_size, n_heads, trg_len, src_len]
        return trg, attention


전체 Decoder
- decoder도 incoder와 같이 반복적으로 수행

In [155]:
class Decoder(nn.Module):
    def __init__(self,output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_rate, device, max_length=100):
        super().__init__()

        self.device = device
        self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([Decoderlayer(hidden_dim,n_heads,pf_dim,dropout_rate,device) for _ in range(n_layers)])
        self.fc_out= nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)
    def forward(self,trg, enc_src, trg_mask, src_mask):
        # trg = [batch_size, trg_len]
        # enc_src = [batch_size, src_len, hidden_dim]
        # trg_mask = [batch_size, trg_len]
        # src_mask = [batch_size, src_len]

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        posision= torch.arange(0,trg_len).unsqueeze(0).repeat(batch_size,1).to(self.device)
        #posision = [batch_size, trg_len]

        trg = self.tok_embedding(trg)* self.scale
        trg = trg + self.pos_embedding(posision)
        trg = self.dropout(trg)
        # trg [batch_size, trg_len, hidden_dim]

        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
            # trg [batch_size, trg_len, hidden_dim]
            # attention = [batch_size, n_heads, trg_len, src_len]

        output = self.fc_out(trg)

        return output,attention


전체 Transformer / encoder+decoder   
- encoder와 decoder를 거쳐 최종 output 문장을 생성합니다 

In [156]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device= device
    
    # Src 문장에서 <pad> 에 대한 문자에 대해 마스크 값을 0으로 설정
    def src_mask(self,src):
        src_mask = (src !=self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask = [batch_size, 1, 1, src_len]
        return src_mask
    # trg 문장에서 다음 단어가 무엇인지 알 수 없도록 마스크 사용
    def trg_mask(self,trg):
        # 마스크 예시, 하나씩 1로 바꾼다.
        # 1 0 0 0 0
        # 1 1 0 0 0
        # 1 1 1 0 0
        # 1 1 1 1 0
        # 1 1 1 1 1

        # <pad>인 것들을 0으로 변경
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        # trg_pad_mask = [batch_size, 1, 1, trg_len]

        trg_len = trg.shape[1]
        # 예시와 같은 sub mask 생성
        trg_sub_mask = torch.tril(torch.ones((trg_len,trg_len), device=self.device)).bool()
        # trg_pad_mask = [trg_len, trg_len]

        # 두개의 mask를 and 연산 bitwise 
        trg_mask = trg_pad_mask & trg_sub_mask
        # trg_mask = [batch_size, 1, trg_len. trg_len]
        return trg_mask
    def forward(self, src, trg):
        # src [batch_size , src_len]
        # trg [batch_size , trg_len]

        src_mask = self.src_mask(src)
        trg_mask = self.trg_mask(trg)

        # src_mask [batch_size, 1, 1, src_len]
        # trg_mask [batch_size, 1, trg_len, trg_len]

        enc_src = self.encoder(src, src_mask)
        # enc_src [batch_size, src_len, hidden_dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        # output = [batch_size, trg_len, output_dim]
        # attention = [batch_size, n_heads, trg_len, src_len]
        
        return output, attention

Training 

In [157]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HIDDEN_DIM = 256
ENC_LAVERS = 3
DEC_LABERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

# encoder decoder 객체 선언
encoder = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAVERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
decoder = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LABERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 선언
model = Transformer(encoder, decoder, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)


def sum_parameter(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Model trainable parameters : {sum_parameter(model)}')

Model trainable parameters : 9038341


In [158]:
# 가중치 초기화
def init_weights(model):
    # hasattr(object, name) , object 에서 name의 속성 확인
    if hasattr(model,'weight') and model.weight.dim() > 1:
        nn.init.xavier_uniform_(model.weight.data)

model.apply(init_weights)

Transformer(
  (encoder): Encoder(
    (tok_embedding): Embedding(7853, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attention_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttention(
          (fc_Q): Linear(in_features=256, out_features=256, bias=True)
          (fc_K): Linear(in_features=256, out_features=256, bias=True)
          (fc_V): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (position_feedforward): PositionwiseFeedforward(
          (fc1): Linear(in_features=256, out_features=512, bias=True)
          (fc2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout):

In [159]:
import torch.optim as optim

LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 패딩 값에 대해서는 무시
criterion = nn.CrossEntropyLoss(ignore_index= TRG_PAD_IDX)

In [160]:
# train 함수
def train(model, train_iterator, optimizer, criterion, clip):
    model.train()
    running_loss = 0

    for i, batch in enumerate(train_iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()
        
        # 맨마지막 <eos>는 제외 하여 
        output, _ = model(src, trg[:,:-1])
        # output = [batch_size, trg_len-1, output_dim]
        # trg = [batch_size, trg_len]

        output_dim = output.shape[-1]
        output = output.contiguous().view(-1, output_dim)
        # 일렬로 쭉 펴기
        # output = [batch_size * trg_len-1, output_dim]

        # trg에서 <sos>는 제외
        trg = trg[:,1:].contiguous().view(-1)
        # trg = [batch_size * trg_len-1]

        loss = criterion(output,trg)
        loss.backward()

        # gradient clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(train_iterator)

In [161]:
# evaluate 함수
def evaluate(model, val_iterator, criterion):
    model.eval()
    val_loss=  0

    with torch.no_grad():
        for i, batch in enumerate(val_iterator):
            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)

            trg = trg[:,1:].contiguous().view(-1)

            loss = criterion(output, trg)

            val_loss += loss.item()

    return val_loss / len(val_iterator)

In [162]:
import time
import math
import random

EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')
history = {}
train_loss_list = []
val_loss_list = []
train_PPL =[]
val_PPL = []
for epoch in range(EPOCHS):
    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(),'./models/Transformer_de_to_en.pt')
    print(f'Epoch : {epoch+1} | Time : {end_time-start_time:.1f}s')
    print(f'\tTrain Loss : {train_loss:.3f} | Train PPL : {math.exp(train_loss):.3f}')
    print(f'\tVal Loss : {valid_loss:.3f} | Val PPL : {math.exp(valid_loss):.3f}')
    train_loss_list.append(train_loss)
    val_loss_list.append(valid_loss)
    train_PPL.append(math.exp(train_loss))
    val_PPL.append(math.exp(valid_loss))


Epoch : 1 | Time : 13.9s
	Train Loss : 4.289 | Train PPL : 72.925
	Val Loss : 3.105 | Val PPL : 22.315
Epoch : 2 | Time : 13.9s
	Train Loss : 2.960 | Train PPL : 19.297
	Val Loss : 2.503 | Val PPL : 12.215
Epoch : 3 | Time : 13.8s
	Train Loss : 2.482 | Train PPL : 11.965
	Val Loss : 2.248 | Val PPL : 9.466
Epoch : 4 | Time : 13.8s
	Train Loss : 2.194 | Train PPL : 8.971
	Val Loss : 2.107 | Val PPL : 8.226
Epoch : 5 | Time : 13.9s
	Train Loss : 1.988 | Train PPL : 7.303
	Val Loss : 2.023 | Val PPL : 7.560
Epoch : 6 | Time : 13.9s
	Train Loss : 1.830 | Train PPL : 6.235
	Val Loss : 1.952 | Val PPL : 7.040
Epoch : 7 | Time : 13.9s
	Train Loss : 1.700 | Train PPL : 5.475
	Val Loss : 1.926 | Val PPL : 6.860
Epoch : 8 | Time : 13.9s
	Train Loss : 1.590 | Train PPL : 4.904
	Val Loss : 1.905 | Val PPL : 6.721
Epoch : 9 | Time : 13.9s
	Train Loss : 1.495 | Train PPL : 4.461
	Val Loss : 1.904 | Val PPL : 6.711
Epoch : 10 | Time : 13.9s
	Train Loss : 1.413 | Train PPL : 4.109
	Val Loss : 1.907 | 

In [163]:
history= {'Train Loss':train_loss_list, 'Val Loss':val_loss_list,'Train PPL':train_PPL,'Val PPL':val_PPL}

In [164]:
model.load_state_dict(torch.load('./models/Transformer_de_to_en.pt'))

test_loss = evaluate(model,test_iterator, criterion)

print(f'Test Loss : {test_loss:.3f} | Test PPL : {math.exp(test_loss):.3f}')

Test Loss : 1.932 | Test PPL : 6.902


In [165]:
def example_translate(sentence, src_field, trg_field, model, device, max_len=50, logging=True):
    model.eval()

    # str 형이라면 sentence가
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    
    tokens = [src_field.init_token]+tokens+[src_field.eos_token]

    if logging:
        print(f'전체 src tokens : {tokens}')
    
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    if logging:
        print(f'src 문장 index : {src_indexes}')
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    print(f'src_tensor shaep : {src_tensor.shape}')
    # maks 생성
    src_mask = model.src_mask(src_tensor)
    print(f'src_mask : {src_mask}\n')
    with torch.no_grad():
        encoder_src = model.encoder(src_tensor, src_mask)
    
    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        # 출력 문장에 따른 마스크 생성
        trg_mask = model.trg_mask(trg_tensor)

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, encoder_src, trg_mask, src_mask)
            #print(f'output shape : {output.shape}\n')
            # 출력 문장에서 가장 마지막 단어만 사용
            pred_token = output.argmax(2)[:,-1].item()
            print(f'pred_token : {pred_token}')
            trg_indexes.append(pred_token)

            # <eos>를 만나면 종료
            if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
                break
    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:], attention

In [166]:
example_idx = 10

ex_src = vars(test_dataset.examples[example_idx])['src']
ex_trg = vars(test_dataset.examples[example_idx])['trg']

print(f'src 문장 : ',*ex_src)
print(f'trg 문장 : ',*ex_trg)

trans, attention = example_translate(ex_src,SRC,TRG, model, device, logging=True)
print('모델 출력 결과 : ', " ".join(trans))

src 문장 :  eine mutter und ihr kleiner sohn genießen einen schönen tag im freien .
trg 문장 :  a mother and her young song enjoying a beautiful day outside .
전체 src tokens : ['<sos>', 'eine', 'mutter', 'und', 'ihr', 'kleiner', 'sohn', 'genießen', 'einen', 'schönen', 'tag', 'im', 'freien', '.', '<eos>']
src 문장 index : [2, 8, 364, 10, 134, 70, 624, 565, 19, 780, 200, 20, 88, 4, 3]
src_tensor shaep : torch.Size([1, 15])
src_mask : tensor([[[[True, True, True, True, True, True, True, True, True, True, True,
           True, True, True, True]]]], device='cuda:0')

pred_token : 4
pred_token : 496
pred_token : 11
pred_token : 44
pred_token : 707
pred_token : 737
pred_token : 4
pred_token : 999
pred_token : 184
pred_token : 341
pred_token : 5
pred_token : 3
모델 출력 결과 :  a mother and her son enjoy a nice day outdoors . <eos>
